In [1]:
from importlib import reload
import sys, csv, json
reload(sys)
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [2]:
import requests

In [3]:
# Below part is not required
########## News API ########################################################
import newsapi

 #key = ('96af62a035db45bda517a9ca62a25ac3')
 #params = {}
 #api = NewsAPI(key)
 #sources = api.sources(params)
 #articles = api.articles(sources[0]['id'], params)

In [4]:
class APIKeyException(Exception):
    def __init__(self, message): self.message = message 

class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message 

class ArchiveAPI(object):
    def __init__(self, key):
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.key = key
        #https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey
        self.root = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None,  key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()

# Replace below key with your NYTimes Developer key
api = ArchiveAPI('YzUwoSmqiVR02u9SSoaX1MNO9YdZwDXa')

years = [2016, 2015, 2014, 2013]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for year in years:
    for month in months:
        mydict = api.query(year, month)
        file_str = '/Users/Ujash/Documents/StockPredictionJPM/NewsData/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()


In [5]:
from bs4 import BeautifulSoup
import datetime
import pickle
import requests

FNAME = "snp500_formatted.txt"
stocks = []

def getNewsForDate(date):
    file = open('data/news/' + date.strftime('%Y-%m-%d') + '.csv', 'w')
    print('Getting news for ' + date.strftime('%Y-%m-%d'))
    for i in range(len(stocks)):
        query = 'http://www.reuters.com/finance/stocks/companyNews?symbol=' + stocks[i] + '&date=' + format(date.month, '02d') + format(date.day, '02d') + str(date.year)
        print('Getting news for ' + stocks[i])

        response = requests.get(query)
        soup = BeautifulSoup(response.text, "html.parser")
        divs = soup.findAll('div', {'class': 'feature'})
        print('Found ' + str(len(divs)) + ' articles.')

        if(len(divs) == 0):
            continue

        data = u''
        for div in divs:
            data = data.join(div.findAll(text=True))
        file.write(stocks[i] + ',' + data.encode('utf-8').replace('\n', ' '))
        file.write('\n')
    file.close()

def getNews():
    dataHistFile = open('dat.pkl', 'rb')
    dataHist = pickle.load(dataHistFile)
    date = dataHist['last_updated'] + datetime.timedelta(days=1)
    endDate = datetime.date.today()

    while(date <= endDate):
        getNewsForDate(date)
        date += datetime.timedelta(days=1)

    dataHist['last_updated'] = endDate
    dataHistFile.seek(0)
    pickle.dump(dataHist, dataHistFile, protocol = pickle.HIGHEST_PROTOCOL)
    dataHistFile.close()

def init():
    global stocks
    with open(FNAME) as f:
        stocks = f.readlines()
    for i in range(len(stocks)):
        stocks[i] = stocks[i].rstrip('\n')

    getNews()

In [28]:
from nytimesarticle import articleAPI
api = articleAPI('YzUwoSmqiVR02u9SSoaX1MNO9YdZwDXa')


In [43]:

import json
import datetime
import time
import sys, os
import logging
from configparser import SafeConfigParser
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2


# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )

# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name

# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
    
# get the articles from the NYTimes Article API    
def getArticles(date, query, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        for n in range(5): # 5 tries
            try:
                request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=" + query + "&begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
                response = urllib2.urlopen(request_string)
                content = response.read()
                if content:
                    articles = convert(json.loads(content))
                    # if there are articles here
                    if len(articles["response"]["docs"]) >= 1:
                        json_file_name = getJsonFileName(date, page, json_file_path)
                        json_file = open(json_file_name, 'w')
                        json_file.write(content)
                        json_file.close()
                    # if no more articles, go to next date
                    else:
                        return
                time.sleep(3) # wait so we don't overwhelm the API
            except HTTPError as e:
                logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
                if e.code == 403:
                    print ("Script hit a snag and got an HTTPError 403. Check your log file for more info.")
                    return
                if e.code == 429:
                    print ("Waiting. You've probably reached an API limit.")
                    time.sleep(30) # wait 30 seconds and try again
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue

# parse the JSON files you stored into a tab-delimited file
def parseArticles(date, tsv_file_name, json_file_path):

    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(date,file_number, json_file_path)
            if os.path.isfile(file_name):
                in_file = open(file_name, 'r')
                articles = convert(json.loads(in_file.read()))
                in_file.close()
            else:
                break
        except IOError as e:
            logging.error("IOError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
            continue
        
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  

            # open the tsv for appending
            try:
                out_file = open(tsv_file_name, 'ab')

            except IOError as e:
                logging.error("IOError: %s %s %s %s", date, file_number, e.errno, e.strerror)
                continue
        
            # loop through the articles putting what we need in a tsv   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
    
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = "\t".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue
        
            out_file.close()
        else:
            break
        
# Main function where stuff gets done

def main():
    
    config = SafeConfigParser()
    script_dir = os.path.dirname(__file__)
    config_file = os.path.join(script_dir, 'config/settings.cfg')
    config.read(config_file)
    
    json_file_path = config.get('files','json_folder')
    tsv_file_name = config.get('files','tsv_file')
    log_file = config.get('files','logfile')
    
    api_key = config.get('nytimes','api_key')    
    start = datetime.date( year = int(config.get('nytimes','start_year')), month = int(config.get('nytimes','start_month')), day = int(config.get('nytimes','start_day')) )
    end = datetime.date( year = int(config.get('nytimes','end_year')), month = int(config.get('nytimes','end_month')), day = int(config.get('nytimes','end_day')) )
    query = config.get('nytimes','query')
        
    logging.basicConfig(filename=log_file, level=logging.INFO)
    
    logging.info("Getting started.") 
    try:
        # LOOP THROUGH THE SPECIFIED DATES
        for date in daterange( start, end ):
            date = date.strftime("%Y%m%d")
            logging.info("Working on %s." % date)
            getArticles(date, query, api_key, json_file_path)
            parseArticles(date, tsv_file_name, json_file_path)
    except:
        logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
    finally:
        logging.info("Finished.")

if __name__ == '__main__' :
    main()

/Users/ujash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:153: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.


NameError: name '__file__' is not defined